# Sequential Agent

This example shows how to create a simple sequential agent to perform a chain of subtasks by agents.

In [1]:
import sys
# Change root directory
sys.path.append("../../src/")

In [2]:
from agent_design_pattern.agent import AgentMessage, LLMChain, BaseAgent
from agent_design_pattern.orchestration import SequentialAgent
from chain import CasualOllamaSingleTurnChain

## Define the agent

Create a simple agent for demonstration

In [3]:
class FunctionImplementationAgent(BaseAgent):
    def __init__(self, llm_chain: LLMChain, state_change_callback = None, name = None, **kwargs):
        super().__init__(state_change_callback, name, **kwargs)
        self.llm_chain = llm_chain

    def execute(self, message: AgentMessage, **kwargs):
        return self.llm_chain.invoke(message, **kwargs)

In [4]:
code_system_prompt = """You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Just the implementation, DO NOT comment anything else"""
code_user_prompt = "{query}"
code_chain = CasualOllamaSingleTurnChain("codegemma:2b-v1.1-q8_0-16384", code_system_prompt, code_user_prompt, base_url="192.168.55.1:11434", request_timeout=300, context_window=32768)

doc_system_prompt = """You are a great developer avocate.
You task is to write a document and in-code explain for the function.
Just the comment and explanation, DO NOT change the logic of the provided function.
The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code"""
doc_user_prompt = """Input query: {query}

Function implementation: {response}
"""
doc_chain = CasualOllamaSingleTurnChain("mistral:7b-instruct-v0.3-q4_K_S", doc_system_prompt, doc_user_prompt, base_url="192.168.55.1:11434", request_timeout=300, context_window=32768)

review_system_prompt = """You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain code, inline code comment and docstring, nothing else."""
review_user_prompt = """Input query: {query}

Function implementation: {response}
"""
review_chain = CasualOllamaSingleTurnChain("qwen3:4b-thinking-2507-q4_K_M", review_system_prompt, review_user_prompt, base_url="192.168.55.1:11434", request_timeout=300, context_window=32768)

def state_callback(message: str):
    print(message)

code_agent = FunctionImplementationAgent(code_chain, state_change_callback=state_callback, name="Code Agent")
doc_agent = FunctionImplementationAgent(doc_chain, state_change_callback=state_callback, name="Docstring Agent")
review_agent = FunctionImplementationAgent(review_chain, state_change_callback=state_callback, name="Review Agent")
agent = SequentialAgent([code_agent, doc_agent, review_agent], state_change_callback=state_callback, name="Sequential Agent")

In [5]:
query = "Write a python function to find the biggest but not exceed the given integer number. The found number must be a number in Fibonacci array."
message = agent.execute(AgentMessage(query=query))

print(message)
print(message.response)

running agent Code Agent
running agent Docstring Agent
running agent Review Agent
idle
query='Write a python function to find the biggest but not exceed the given integer number. The found number must be a number in Fibonacci array.' origin='Sequential Agent' response='\n\n```python\ndef find_biggest_fibonacci(limit, fibonacci=[0, 1]):\n    """\n    Returns the biggest Fibonacci number that is less than or equal to the given limit.\n\n    Args:\n        limit (int): The maximum integer value to consider.\n        fibonacci (list, optional): A predefined Fibonacci sequence. Defaults to [0, 1].\n\n    Returns:\n        int: The biggest Fibonacci number that is less than or equal to the given limit.\n\n    Example:\n        >>> find_biggest_fibonacci(10)\n        8\n    """\n    # Generate Fibonacci numbers until the next number exceeds the limit\n    while fibonacci[-1] <= limit:\n        next_fib = fibonacci[-1] + fibonacci[-2]\n        fibonacci.append(next_fib)\n    \n    return fibon